In [ ]:
from facebook_business.api import FacebookAdsApi
from facebook_business import adobjects
from facebook_business.adobjects.adaccountuser import AdAccountUser
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.adobjects.adreportrun import AdReportRun
import pandas as pd

my_app_id = 'xxx'
my_app_secret = 'xxx'
my_access_token = 'xxx'
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

me = AdAccountUser(fbid='me')
adAccounts = me.get_ad_accounts()
me.get_ad_accounts()

account = AdAccount('act_466052887236542')
ads = account.get_ads()

sdate=''
edate=''
print("Введи дату начала периода в формате 2019-04-01")
sdate=str(input())
print("Введи дату конца периода в формате 2019-04-30")
edate=str(input())
params = {
    'time_range': {
        'since':  "{}".format(sdate), 
        'until': "{}".format(edate)
        }, 
    'fields': [
        AdsInsights.Field.campaign_id,
        AdsInsights.Field.campaign_name,
        AdsInsights.Field.adset_name,
        AdsInsights.Field.ad_name,
        AdsInsights.Field.spend,
        AdsInsights.Field.impressions,
        AdsInsights.Field.clicks,
        AdsInsights.Field.buying_type,
        AdsInsights.Field.objective,        
        AdsInsights.Field.actions,
        AdsInsights.Field.cpm,
        AdsInsights.Field.account_currency,
     ],
    'breakdowns': ['publisher_platform'],
    'level': 'ad',
    'time_increment': 'all_days',
}


In [ ]:
async_job = account.get_insights_async(fields = params.get('fields'), params=params)

In [ ]:
result = [x for x in async_job.get_result()]
df_res = pd.DataFrame(result)

all_convs = list(df_res['actions'])
conv1 = []
for e in all_convs:
    if str(e) == 'nan':
        conv1.append(0)
    else:
        test_val=[]
        for m in e:
            for key, val in m.items():
                if val == 'offsite_conversion.custom.1598324666980073':
                    conv1.append(int(m['value']))
                    test_val.append(int(m['value']))
        if len(test_val) == 0:
            conv1.append(0)

conv2 = []
for e in all_convs:
    if str(e) == 'nan':
        conv2.append(0)
    else:
        test_val=[]
        for m in e:
            for key, val in m.items():
                if val == 'offsite_conversion.custom.2245672202417987':
                    conv2.append(int(m['value']))
                    test_val.append(int(m['value']))
        if len(test_val) == 0:
            conv2.append(0) 

df_res['request_send_all'] = conv1
df_res['regconf'] = conv2

df_res[['account_id', 'ad_name', 'adset_name', 'campaign_id', 'impressions','clicks', 'spend','cpm','request_send_all','regconf','publisher_platform']].to_excel('Facebook_Report_{}_{}.xlsx'.format(sdate,edate),index=False)
df_res[['account_id', 'ad_name', 'adset_name', 'campaign_id', 'impressions','clicks', 'spend','cpm','request_send_all','regconf','publisher_platform']]